# Dynamic Tropopause Calculation

By: Kevin Goebbert

This example uses MetPy calculation ability to determine the potential temperature on the dynamic tropopause (2 PVU surface), add the derived variables to the xarray dataset and plot using the MetPy declarative syntax.

In [ ]:
from datetime import datetime, timedelta

import metpy.calc as mpcalc
from metpy.interpolate import interpolate_to_isosurface
from metpy.plots.declarative import *
from metpy.units import units
import xarray as xr

## Get GFS Data

Obtain and subset GFS data to cover the CONUS region

In [ ]:
date = datetime.utcnow() - timedelta(days=1)
ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/'
                     f'Global_onedeg_ana/GFS_Global_onedeg_ana_{date:%Y%m%d}'
                     '_1200.grib2').metpy.parse_cf()

ds = ds.sel(lat=slice(80, -10), lon=slice(360-140, 360-40))

vtime = ds.time.values[0].astype('datetime64[ms]').astype('O')

## Compute Potential Temperature at 2 PVU
The following cell takes the necessary data from the GFS analysis, smooths and calculates needed variables to obtain the potential temperature on the 2 PVU surface (e.g., the dynamic tropopause), as well as interpolate the wind components to that level as well.

In [ ]:
pressure = ds.isobaric

uwind = mpcalc.smooth_n_point(
    ds['u-component_of_wind_isobaric'].squeeze(), 9, 2)
vwind = mpcalc.smooth_n_point(
    ds['v-component_of_wind_isobaric'].squeeze(), 9, 2)

LL_avor = mpcalc.smooth_n_point(ds.Absolute_vorticity_isobaric.metpy.sel(
    vertical=slice(850 * units.hPa, 925 * units.hPa)).squeeze(), 9, 2)
avg_LL_avor = LL_avor.mean(axis=0)

potemp = mpcalc.smooth_n_point(
    mpcalc.potential_temperature(pressure,
                                 ds.Temperature_isobaric.squeeze()), 9, 2)
avg_LL_rvor = avg_LL_avor - mpcalc.coriolis_parameter(ds.lat)

pvor = mpcalc.potential_vorticity_baroclinic(potemp, pressure, uwind, vwind)

DT_potemp = interpolate_to_isosurface(pvor.values*1e6, potemp.values, 2,
                                      bottom_up_search=False)
DT_uwnd = (interpolate_to_isosurface(pvor.values*1e6, uwind.values, 2,
                                     bottom_up_search=False) * units('m/s')).to(units.knots)
DT_vwnd = (interpolate_to_isosurface(pvor.values*1e6, vwind.values, 2,
                                     bottom_up_search=False) * units('m/s')).to(units.knots)

## Add Variables to Dataset
This next cell adds the variables calculated/derived above to the xarray dataset, which will make them available for plotting with the MetPy declarative syntax.

In [ ]:
# Assign values that are not DataArrays
gridmapping = ds['u-component_of_wind_isobaric'].grid_mapping
ds = ds.assign(dynamic_trop=(tuple(('lat', 'lon')), DT_potemp,
               {'grid_mapping': gridmapping, 'units': 'K'}))
ds = ds.assign(uwnd_DT=(tuple(('lat', 'lon')), DT_uwnd.m,
               {'grid_mapping': gridmapping, 'units': 'knots'}))
ds = ds.assign(vwnd_DT=(tuple(('lat', 'lon')), DT_vwnd.m,
               {'grid_mapping': gridmapping, 'units': 'knots'}))

# Assign values that are DataArrays
ds['avg_LL_rel_vort'] = avg_LL_rvor

## Create the Plot

In [ ]:
cntr = ContourPlot()
cntr.data = ds
cntr.level = None
cntr.field = 'avg_LL_rel_vort'
cntr.clabels = True
cntr.contours = [0.5, 1.5, 2.5, 3.5, 4.5]
cntr.scale = 1e4

cntr2 = FilledContourPlot()
cntr2.data = ds
cntr2.level = None
cntr2.field = 'dynamic_trop'
cntr2.contours = list(range(250, 420, 1))
cntr2.colormap = 'coolwarm'
cntr2.colorbar = 'horizontal'

barbs = BarbPlot()
barbs.data = ds
barbs.field = ['uwnd_DT', 'vwnd_DT']
barbs.skip = (3, 3)

panel = MapPanel()
panel.projection = 'lcc'
panel.area = 'us'
panel.layers = ['states', 'borders', 'coastline']
panel.title = ('Dynamic Tropopause Potential Temperature (K),'
               'Wind Barbs (kts), and LL Rel. Vort. (s$^{-1}$) at '
               f'{vtime}')
panel.plots = [cntr2, cntr, barbs]

pc = PanelContainer()
pc.size = (18, 14)
pc.panels = [panel]

pc.show()